In [2]:
from six.moves import cPickle as pickle
import numpy as np
import os
from scipy.misc import imread, imshow

In [3]:
def load_pickle(f):
    return  pickle.load(f, encoding='latin1')
    
def load_CIFAR_batch(filename):
    """ load single batch of cifar """
    with open(filename, 'rb') as f:
        datadict = load_pickle(f)
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
        Y = np.array(Y)
    return X, Y

def load_CIFAR10(ROOT):
    """ load all of cifar """
    xs = []
    ys = []
    for b in range(1,6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)    
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte

In [4]:
xTrain, yTrain, xTest, yTest = load_CIFAR10("Datasets/cifar-10-batches-py/")
print("X_train:", xTrain.shape)
print("Y_train:", yTrain.shape)
print("X_test:", xTest.shape)
print("Y_test:", yTest.shape)

X_train: (50000, 32, 32, 3)
Y_train: (50000,)
X_test: (10000, 32, 32, 3)
Y_test: (10000,)


In [5]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# extract features

In [6]:
from skimage.feature import hog
from skimage.color import rgb2gray
from skimage import data, exposure
from skimage.feature import local_binary_pattern

def extract_features(mode, X_train, X_test):
    if mode == "RAW":
        X_train = X_train
        X_test = X_test
    elif mode == "HOG":
        X_train = np.array([hog(rgb2gray(image), orientations=8, pixels_per_cell=(16, 16),
                   cells_per_block=(1, 1)) for image in X_train])
        X_test = np.array([hog(rgb2gray(image), orientations=8, pixels_per_cell=(16, 16),
                   cells_per_block=(1, 1)) for image in X_test])
    elif mode == "LBP":
        X_train = np.array([local_binary_pattern(rgb2gray(image),P=1,R=2) for image in X_train])
        X_test = np.array([local_binary_pattern(rgb2gray(image),P=1,R=2) for image in X_test])
    elif mode == "SIFT":
        def sift(img):
            gray= cv2.cvtColor(img.astype(np.uint8),cv2.COLOR_BGR2GRAY)  
            sift = cv2.xfeatures2d.SIFT_create(18)
            kp = sift.detect(gray,None)  
            des = sift.compute(gray,kp)
            return des[-1]
        X_train = np.array([sift(image) for image in X_train])
        X_test = np.array([sift(image) for image in X_test])
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    return X_train, X_test


# models

In [48]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [49]:
class Cifar10(Dataset):
    def __init__(self, data, label, data_num):
        super(Cifar10, self).__init__()
        self.data_num = data_num
        self.data = data
        self.label = label
    
    def __getitem__(self, index):
        data = self.data[index].astype(np.float32)
        label = self.label[index]
        return data, label
    
    def __len__(self):
        return self.data_num


In [70]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(3072, 3072),
            nn.LeakyReLU(inplace=True),
            nn.Linear(3072, 1024),
            nn.LeakyReLU(inplace=True),
            nn.Linear(1024, 256),
            nn.LeakyReLU(inplace=True),
            nn.Linear(256, 10)
        )
    
    def forward(self, data):
        output = self.model(data)
        return output
    

# train and test

In [71]:
NUM_TRAIN = 50000
NUM_TEST = 10000
BATCH_SIZE = 256
LEARNING_RATE = 0.001

trainDataset = Cifar10(xTrain, yTrain, NUM_TRAIN)
testDataset = Cifar10(xTest, yTest, NUM_TEST)
trainLoader = DataLoader(dataset=trainDataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
testLoader = DataLoader(dataset=testDataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
model = MLP()
model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [72]:
%%time
NUM_EPOCHS = 50
for epoch in range(NUM_EPOCHS):
    train_loss = 0
    test_loss = 0
    train_acc = 0
    test_acc = 0
    for i, (data, label) in enumerate(trainLoader):
        data = Variable(data.view(-1, 3072)).cuda()
        label = Variable(label.view(-1)).cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, label)
        loss.backward()
        train_loss += loss.data[0]
        _, predict = torch.max(output, 1)
        num_correct = (predict == label).sum()
        train_acc += num_correct.data[0]
        optimizer.step()
        
    for i, (data, label) in enumerate(testLoader):
        data = Variable(data.view(-1, 3072)).cuda()
        label = Variable(label.view(-1)).cuda()
        output = model(data)
        loss = criterion(output, label)
        test_loss += loss.data[0]
        _, predict = torch.max(output, 1)
        num_correct = (predict == label).sum()
        test_acc += num_correct.data[0]
        
    print('Epoch [%d/%d], Train Loss: %.4f, Train Acc: %.4f, Test Loss: %.4f, Test Acc: %.4f'
            %(epoch+1, NUM_EPOCHS, 
              train_loss / NUM_TRAIN, train_acc / NUM_TRAIN, 
              test_loss / NUM_TEST, test_acc / NUM_TEST))

Epoch [1/50], Train Loss: 0.0798, Train Acc: 0.2038, Test Loss: 0.0076, Test Acc: 0.2803
Epoch [2/50], Train Loss: 0.0073, Train Acc: 0.3200, Test Loss: 0.0070, Test Acc: 0.3614
Epoch [3/50], Train Loss: 0.0070, Train Acc: 0.3606, Test Loss: 0.0069, Test Acc: 0.3712
Epoch [4/50], Train Loss: 0.0067, Train Acc: 0.3821, Test Loss: 0.0065, Test Acc: 0.4099
Epoch [5/50], Train Loss: 0.0066, Train Acc: 0.4012, Test Loss: 0.0065, Test Acc: 0.3984
Epoch [6/50], Train Loss: 0.0064, Train Acc: 0.4178, Test Loss: 0.0064, Test Acc: 0.4245
Epoch [7/50], Train Loss: 0.0063, Train Acc: 0.4270, Test Loss: 0.0061, Test Acc: 0.4427
Epoch [8/50], Train Loss: 0.0061, Train Acc: 0.4403, Test Loss: 0.0063, Test Acc: 0.4229
Epoch [9/50], Train Loss: 0.0061, Train Acc: 0.4489, Test Loss: 0.0060, Test Acc: 0.4533
Epoch [10/50], Train Loss: 0.0060, Train Acc: 0.4529, Test Loss: 0.0061, Test Acc: 0.4394
Epoch [11/50], Train Loss: 0.0058, Train Acc: 0.4651, Test Loss: 0.0060, Test Acc: 0.4501
Epoch [12/50], Trai